In [27]:
import recordlinkage
from Annotation_helper import *
import numpy as np

In [28]:
data_folder = "../../Data"
structured_data_folder = data_folder + "/structured_data"
text_path = data_folder + "/text"
data_path = structured_data_folder + "/test.csv"
dist_folder = structured_data_folder + "/filtered_Distances"
figure_folder = "../Figures/Results/Embeddings"
model_folder = "../Models/Embeddings"
matches = pd.read_csv(structured_data_folder + "/Matches/Bas3.csv", index_col=0)
figure_folder = "../Figures/Distances"

In [29]:
transactions = load_transactions("../../Data/structured_data\VOC Cochin Slave Transactions 1706-1801 - IISH - Database - 2018 - v1.csv")
permissions = load_permissions("../../Data/structured_data\VOC Cochin Slave Transport Permissions 1770-1795 - IISH - Database - 2018 - v1.csv")

In [30]:
transactions.columns

Index(['Unnamed: 0', 'ID', 'Datum', 'VerkoperVoornaam', 'VerkoperPatroniem',
       'VerkoperTussen', 'VerkoperAchternaam', 'VerkoperGender',
       'VerkoperHerkomst', 'VerkoperBeroep', 'VerkoperWerkplek',
       'VerkoperEtniciteit', 'VerkoperStatus', 'KoperVoornaam',
       'KoperPatroniem', 'KoperTussen', 'KoperAchternaam', 'KoperGender',
       'KoperHerkomst', 'KoperBeroep', 'KoperWerkplek', 'KoperEtniciteit',
       'KoperStatus', 'SlaafNaamNieuw', 'SlaafNaamOrigineel', 'SlaafTaalNaam',
       'SlaafKaste', 'SlaafGender', 'SlaafLeeftijd', 'Opmerking', 'Entry-ID'],
      dtype='object')

In [31]:
permissions.columns

Index(['ID', 'Permissie', 'Inventarisnummer', 'Scan', 'Plaats', 'Datum',
       'BezitterVoornaam', 'BezitterPatroniem', 'BezitterTussen',
       'BezitterAchternaam', 'BezitterGender', 'BezitterHerkomst',
       'BezitterBeroep', 'BezitterWerkplek', 'BezitterEtniciteit',
       'BezitterStatus', 'SlaafNaamNieuw', 'SlaafNaamOrigineel', 'SlaafKaste',
       'SlaafGender', 'Opmerking', 'Betaling', 'Door[Kantlijn]'],
      dtype='object')

In [32]:
permissions[permissions["BezitterEtniciteit"].notna()]

,ID,Permissie,Inventarisnummer,Scan,Plaats,Datum,BezitterVoornaam,BezitterPatroniem,BezitterTussen,BezitterAchternaam,...,BezitterWerkplek,BezitterEtniciteit,BezitterStatus,SlaafNaamNieuw,SlaafNaamOrigineel,SlaafKaste,SlaafGender,Opmerking,Betaling,Door[Kantlijn]
24,897-1-25,25,897,1,Cochin,[…]-05-09,David,NaN,NaN,More,...,Schip Peintroke,Engels,NaN,Joan,NaN,NaN,m,NaN,M[its betalende ge]regtigheid aan den pagter,den vaandrig der burgerije M: Lavier
106,897-5-8,8,897,5,Cochin,1770-10-29,Juhn,NaN,NaN,Meken,...,NaN,Engelsch,NaN,October,NaN,NaN,m,NaN,NaN,NaN
107,897-5-9,9,897,5,Cochin,1770-10-29,Juhn,NaN,NaN,Meken,...,NaN,Engelsch,NaN,October,NaN,NaN,m,NaN,NaN,NaN
108,897-5-10,10,897,5,Cochin,1770-11-06,Paxsou,NaN,NaN,NaN,...,NaN,Engelsch,NaN,Roza,NaN,NaN,v,NaN,Enz:,Den herbergier Groenroode gekogt
109,897-5-11,11,897,5,Cochin,1770-11-06,Kepp,NaN,NaN,NaN,...,NaN,Engelsch,NaN,Thome,NaN,NaN,m,NaN,Enz:,Daniel Bakker verkogt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,1333-242-3,3,1333,242,Cochin,1795-05-01,NaN,NaN,NaN,NaN,...,NaN,Deens,NaN,Roza,NaN,NaN,v,NaN,Gerechtigheid,NaN
3920,1333-242-4,4,1333,242,Cochin,1795-05-01,Douglas,NaN,NaN,NaN,...,NaN,Engels,NaN,Antoni,NaN,NaN,m,NaN,Gerechtigheid,['...]
3922,1333-243-1,1,1333,243,Cochin,1795-05-01,Jean,NaN,NaN,Jasjean,...,NaN,Frans,NaN,Dalida,NaN,NaN,v,NaN,Gerechtigheid,NaN
3923,1333-243-2,2,1333,243,Cochin,1795-05-04,Douglas,NaN,NaN,NaN,...,NaN,Engels,NaN,Flo[...],NaN,NaN,NaN,NaN,Gerechtigheid,NaN


In [33]:
permissions["BezitterEtniciteit"].unique()

array([nan, 'Engels', 'Engelsch', '[Engel]sch', 'Jood', 'Fransch',
       'Portugees', 'Armeender', 'Engelsche', 'Moor', 'Engelsman',
       'Maleijs', 'Toepas', 'Joods', 'Armeens', 'Deens', 'Portugeesch',
       'Frans', 'Deen', 'Englesch', 'Armenijer', 'Armeniers', 'Arabier',
       '[Fransch]', 'Armeensch', '[Por]tugeesch', 'Armeendersch',
       'Fransman', 'Deensch', 'Italiaansch', 'Moorsch', 'Malaitsch',
       'Het schip Teijlingen'], dtype=object)

In [34]:
dist_metric = 'levenshtein'
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(permissions, transactions)

compare_cl = recordlinkage.Compare()

compare_cl.exact('SlaafGender', 'SlaafGender', label='SlaafGender')
compare_cl.string('SlaafNaamNieuw', 'SlaafNaamNieuw', label='SlaafNaamNieuw', method=dist_metric)
compare_cl.exact('BezitterGender', 'KoperGender', label='BezitterGender')
compare_cl.string('BezitterVoornaam', 'KoperVoornaam', label='BezitterVoornaam', method=dist_metric)
compare_cl.string('BezitterTussen', 'KoperTussen', label='BezitterTussen', method=dist_metric)
compare_cl.string('BezitterAchternaam', 'KoperAchternaam', label='BezitterAchternaam', method=dist_metric)
compare_cl.string('BezitterBeroep', 'KoperBeroep', label='BezitterBeroep', method=dist_metric)
compare_cl.string('BezitterStatus', 'KoperStatus', label='BezitterStatus', method=dist_metric)
compare_cl.string('BezitterEtniciteit', 'KoperEtniciteit', label='BezitterEtniciteit', method=dist_metric)
n_batches = 1
split_perm = np.array_split(permissions, n_batches)

In [35]:
for i, p in enumerate(split_perm):
    print(f"[{i}/{n_batches}]")
    candidate_links = indexer.index(p, transactions)
    features = compare_cl.compute(candidate_links, p, transactions)
    features = features.reset_index()
    features = features.rename(columns={"level_0": "permission_indx", "level_1": "transaction_indx"})
    features.to_csv(f"{dist_folder}_{i}.csv", index=False)
    clear_output(wait=True)

[0/1]


MemoryError: Unable to allocate 1.80 GiB for an array with shape (9, 26904684) and data type object

In [ ]:
distances = pd.read_csv(dist_folder + "/distances_0.csv", dtype={'permission_indx': 'uint16', 'transaction_indx': 'uint16', 'SlaafGender': 'bool', 'SlaafNaamNieuw': 'float16', 'BezitterGender': 'bool', 'BezitterVoornaam': 'float16', 'BezitterBeroep': 'float16', 'BezitterAchternaam': 'float16'})
for i in range(1, 99):
    df2 = pd.read_csv(f"{dist_folder}/distances_{i}.csv", dtype={'permission_indx': 'uint16', 'transaction_indx': 'uint16', 'SlaafGender': 'bool', 'SlaafNaamNieuw': 'float16', 'BezitterGender': 'bool', 'BezitterVoornaam': 'float16', 'BezitterBeroep': 'float16', 'BezitterAchternaam': 'float16'})
    distances = pd.concat([distances, df2])
distances.reset_index(inplace=True)

In [7]:
distances

,index,permission_indx,transaction_indx,SlaafGender,SlaafNaamNieuw,BezitterGender,BezitterVoornaam,BezitterTussen,BezitterAchternaam,BezitterBeroep,BezitterStatus,BezitterEtniciteit
0,0,0,0,True,0.000000,False,0.000000,0.0,0.000000,0.0,0.0,0.0
1,1,0,1,True,0.250000,False,0.000000,0.0,0.000000,0.0,0.0,0.0
2,2,0,2,False,0.166626,False,0.000000,0.0,0.000000,0.0,0.0,0.0
3,3,0,3,True,0.111084,False,0.000000,0.0,0.000000,0.0,0.0,0.0
4,4,0,4,True,0.142822,False,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
26638723,265951,3889,7192,True,0.250000,True,0.166626,0.0,0.166626,0.0,0.0,1.0
26638724,265952,3889,7193,False,0.250000,True,0.166626,0.0,0.166626,0.0,0.0,1.0
26638725,265953,3889,7194,False,0.000000,True,0.277832,0.0,0.000000,0.0,0.0,1.0
26638726,265954,3889,7195,False,0.000000,True,0.166626,0.0,0.166626,0.0,0.0,1.0
